In [1]:
print('hello world')

hello world


In [3]:
import numpy
from matplotlib import pyplot
%matplotlib inline

In [4]:
def year_1859_to_1896(p_initial):
    
    dt = 1    #1 month
    dx = 1    #100 km
    
    
    months = 0
    growth_rate = 0.005
    p_n = numpy.empty_like(p_initial)
    p = p_initial.copy()
    
    
    while months != 444:
        months = 0
        p = p_initial.copy()
        
        
        
        while p[30,30] == 0:
            p_n = p.copy()
            p[1:-1,1:-1] = (1 + growth_rate)*p_n[1:-1,1:-1] + dt/dx**2 * (p_n[1:-1,:-2] + p_n[1:-1,2:] + p_n[:-2,1:-1] + p_n[2:,1:-1] - \
                                                                                      4*p_n[1:-1,1:-1])
            
            p[0,:] = p[1,:]
            p[-1,:] = p[-2,:]
            p[:,0] = p[:,1]
            p[:,-1] = p[:,-2]
            
            
            months = months + 1
            
            
            
        if months > 444:
            growth_rate = growth_rate*1.5
        elif months < 444:
            growth_rate = growth_rate*0.5
            
            
    return p, growth_rate

In [5]:
p_initial = numpy.zeros((37,40))
p_initial[0,30] = 24

In [6]:
p_initial[0]

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  24.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.])

In [ ]:
p_1896, growth_rate_1896 = year_1859_to_1896(p_initial)

In [8]:
growth_rate_1896

NameError: name 'growth_rate_1896' is not defined